# Manufacturer / Aircraft Type Pie Chart

This notebook produces a pie chart of aicraft models for a given manufacturer.

Individual models within a family are grouped:

- e.g. Both A320-214 and A320-232 will appear under the "A320" family
- Aircraft with fewer sightings than the configured threshold are asigned to the "Other" family

To use the notebook, set the manufacturer, threshold and export format for charts in the first code cell before running the notebook.

In [ ]:
# Manufacturer to report for
manufacturer = "Airbus"

# Sightings threshold below which aircraft are assigned to a generic "Other" group. When set to 1, no models will be assigned to the "other" group
prefix_count_threshold = 1

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
export_format = "PNG"

In [ ]:
from pathlib import Path
import sqlparse

# Read the query file
query_file_path = Path("sql") / "manufacturer_sightings.sql"

with open(query_file_path.absolute(), "r") as f:
    query = f.read().replace("\n", " ")

# Replace placeholders
query = query.replace("$MANUFACTURER", manufacturer)

# Show a pretty-printed form of the query
print(sqlparse.format(query, reindent=True, keyword_case="upper"))

In [ ]:
import pandas as pd
import sqlite3
import os

# Connect to the database, execute the query and read the results into a dataframe
database_path = os.environ["FLIGHT_RECORDER_DB"]
connection = sqlite3.connect(database_path)
df = pd.read_sql_query(query, connection, parse_dates=["Date"])

# Check there is some data
if not df.shape[0]:
    message = f"No data found"
    raise ValueError(message)

# Preview the data
df.head()

In [ ]:
import re

def extract_prefix(model):
    # Try extracting leading letters and digits (e.g., "A320", "B737")
    match = re.match(r"^([A-Z]+[0-9]+)", str(model).replace(" ", "").upper())
    prefix = match.group(1) if match else None

    # If this doesn"t result in a prefix, try just splitting on the first "-"
    if not prefix:
        tokens = model.split("-", 1)
        prefix = tokens[0] if len(tokens) > 1 else None

    # On its own, the above can result in some nonsensical prefixes so check we do have some numbers in
    # to indicate the model
    if prefix and not any(char.isdigit() for char in prefix):
        prefix = None

    return prefix

# Extract an aircraft type prefix for each sighting
df["Type"] = df["Model"].apply(extract_prefix)

# Preview the data
df.head()

In [ ]:
# Identify counts by prefix and use them to identify common prefixes
prefix_counts = df["Type"].value_counts()
common_prefixes = prefix_counts[prefix_counts >= prefix_count_threshold].index

# Replace uncommon prefixes with "Other"
df["Type"] = df.apply(
    lambda row: row["Type"] if row["Type"] in common_prefixes else "Other",
    axis=1
)

# Preview the data
df.head()

In [ ]:
# Calculate the % of each aircraft type
group_counts = df["Type"].value_counts()
group_percent = group_counts / group_counts.sum() * 100

In [ ]:
import pandas as pd
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Create a Pandas Excel writer
clean_manufacturer = re.sub("[^0-9a-zA-Z ]+", "", manufacturer).replace(" ", "-")
export_file_name = f"{clean_manufacturer}-Model-Chart"
output_path = export_folder_path / f"{export_file_name}.xlsx"

with pd.ExcelWriter(output_path.absolute(), engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Sightings Data", index=False)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 8))

# Draw pie chart
wedges, texts = ax.pie(
    group_percent,
    startangle=90,
    wedgeprops=dict(width=0.5),
    labels=None,
    counterclock=False
)

# Format legend labels as Type (XX.X%)"
legend_labels = [f"{grp} ({pct:.1f}%)" for grp, pct in zip(group_percent.index, group_percent.values)]

# Add a legend below the chart
ax.legend(
    wedges,
    legend_labels,
    title="Aircraft Groups",
    loc='lower center',
    bbox_to_anchor=(0.5, -0.2),
    ncol=2,
    fontsize=8
)

# Set the title
plt.title(f"{manufacturer} Aircraft Sightings by Group")
plt.tight_layout()


# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()
